In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [8]:
%run -i ../../core.py

In [9]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

Currently, the circuit is too huge, possibly by requesting covariance and stdev and just calculate it on verifier side instead!

In [10]:
x_vals_path = os.path.join('x_vals.json')
dummy_x_vals_path = os.path.join('shared/dummy_x_vals.json')
x_open = open(x_vals_path, "r")
x_vals= json.loads(x_open.read())['input_data'][0]
dummy_x_vals = np.random.uniform(min(x_vals), max(x_vals), len(x_vals))
json.dump({"input_data":[dummy_x_vals.tolist()]}, open(dummy_x_vals_path, 'w'))


y_vals_path = os.path.join('y_vals.json')
dummy_y_vals_path = os.path.join('shared/dummy_y_vals.json')
y_open = open(y_vals_path, "r")
y_vals= json.loads(y_open.read())["input_data"][0]
dummy_y_vals = np.random.uniform(min(y_vals), max(y_vals), len(y_vals))
json.dump({"input_data":[dummy_y_vals.tolist()]}, open(dummy_y_vals_path, 'w'))


real_corr = statistics.correlation(x_vals, y_vals)
x_mean = statistics.mean(x_vals)
y_mean = statistics.mean(y_vals)
x_var = np.sum((np.array(x_vals)-x_mean)**2)
y_var = np.sum((np.array(y_vals)-y_mean)**2)
print("corr: ",real_corr )
print("x mean: ", x_mean)
print("y mean: ", y_mean)

dummy_corr = statistics.correlation(dummy_x_vals, dummy_y_vals)
dummy_x_mean = statistics.mean(dummy_x_vals)
dummy_y_mean = statistics.mean(dummy_y_vals)
dummy_x_var = np.sum((dummy_x_vals-dummy_x_mean)**2)
dummy_y_var = np.sum((dummy_y_vals-dummy_y_mean)**2)
print("dummy corr: ",dummy_corr )
print("dummy x mean: ", dummy_x_mean)
print("dummy y mean: ", dummy_y_mean)

corr:  0.8928769029553791
x mean:  49.5
y mean:  227.703
dummy corr:  0.09636370915922249
dummy x mean:  49.00761737545071
dummy y mean:  237.99976726082184


In [34]:
# precise float number is hard, so we calculate 100*correlation instead.
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.corr = nn.Parameter(data = torch.tensor(dummy_corr*100), requires_grad = False)
        self.x_mean = nn.Parameter(data = torch.tensor(dummy_x_mean), requires_grad = False)
        self.y_mean = nn.Parameter(data = torch.tensor(dummy_y_mean), requires_grad = False)
        self.x_var = nn.Parameter(data = torch.tensor(dummy_x_var), requires_grad = False)
        self.y_var = nn.Parameter(data = torch.tensor(dummy_y_var), requires_grad = False)
    def forward(self,X,Y):
        # some expression of tolerance to error in the inference
        #  need to enforce same length, not yet
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.x_mean))<0.01*X.size()[1]*(self.x_mean)
        y_mean_cons = torch.abs(torch.sum(Y)-Y.size()[1]*(self.y_mean))<0.01*Y.size()[1]*(self.y_mean)
        x_var_cons = torch.abs(torch.sum((X-self.x_mean)*(X-self.x_mean))-self.x_var*(X.size()[1]-1))<0.01*self.x_var*(X.size()[1]-1)
        y_var_cons = torch.abs(torch.sum((Y-self.y_mean)*(Y-self.y_mean))-self.y_var*(Y.size()[1]-1))<0.01*self.y_var*(Y.size()[1]-1)
        corr_cons = torch.abs(torch.sum((X-self.x_mean)*(Y-self.y_mean))-(torch.sqrt(self.x_var)*torch.sqrt(self.y_var)*torch.div(self.corr, 100)))<0.01*(torch.sqrt(self.x_var)*torch.sqrt(self.y_var)*torch.div(self.corr, 100))

        # return (torch.logical_and(corr_cons, torch.logical_and(torch.logical_and(x_mean_cons, y_mean_cons), torch.logical_and(x_var_cons, y_var_cons))), self.corr)
        # return (torch.logical_and(corr_cons, torch.logical_and(x_mean_cons, y_mean_cons)), self.corr)
        # return (torch.logical_and(torch.logical_and(x_var_cons,corr_cons), torch.logical_and(x_mean_cons, y_mean_cons)), self.corr)
        # return (torch.logical_and(x_var_cons,torch.logical_and(corr_cons, torch.logical_and(x_mean_cons, y_mean_cons))), self.corr)
        return (torch.logical_and(x_var_cons, corr_cons), self.corr)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_x_vals_path, dummy_y_vals_path])

xxx:  tensor([[[62.9795],
         [22.0217],
         [52.3409],
         [37.5802],
         [58.1255],
         [52.4249],
         [20.3958],
         [47.2491],
         [23.8496],
         [ 7.8126],
         [13.9089],
         [79.6785],
         [43.2629],
         [62.6596],
         [92.1490],
         [17.4866],
         [14.0891],
         [ 4.9190],
         [78.6948],
         [41.2017],
         [42.2647],
         [43.9585],
         [83.9661],
         [89.5124],
         [95.3961],
         [60.9797],
         [23.4067],
         [17.3428],
         [90.6992],
         [79.7309],
         [87.4945],
         [45.8548],
         [22.1585],
         [92.6558],
         [ 7.5434],
         [ 6.0029],
         [46.7445],
         [59.2296],
         [26.6279],
         [43.9125],
         [ 2.2883],
         [52.6816],
         [35.7346],
         [54.6388],
         [75.4577],
         [ 7.3459],
         [82.0465],
         [85.7411],
         [46.4742],
         [64.1

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [37]:
# Prover/ data owner side
theory_output = torch.tensor(real_corr)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.corr = nn.Parameter(data = torch.tensor(real_corr*100), requires_grad = False)
        self.x_mean = nn.Parameter(data = torch.tensor(x_mean), requires_grad = False)
        self.y_mean = nn.Parameter(data = torch.tensor(y_mean), requires_grad = False)
        self.x_var = nn.Parameter(data = torch.tensor(x_var), requires_grad = False)
        self.y_var = nn.Parameter(data = torch.tensor(y_var), requires_grad = False)
    def forward(self,X,Y):
        # some expression of tolerance to error in the inference
        #  need to enforce same length, not yet
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.x_mean))<0.01*X.size()[1]*(self.x_mean)
        y_mean_cons = torch.abs(torch.sum(Y)-Y.size()[1]*(self.y_mean))<0.01*Y.size()[1]*(self.y_mean)
        x_var_cons = torch.abs(torch.sum((X-self.x_mean)*(X-self.x_mean))-self.x_var*(X.size()[1]-1))<0.01*self.x_var*(X.size()[1]-1)
        y_var_cons = torch.abs(torch.sum((Y-self.y_mean)*(Y-self.y_mean))-self.y_var*(Y.size()[1]-1))<0.01*self.y_var*(Y.size()[1]-1)
        corr_cons = torch.abs(torch.sum((X-self.x_mean)*(Y-self.y_mean))-(torch.sqrt(self.x_var)*torch.sqrt(self.y_var)*torch.div(self.corr, 100)))<0.01*(torch.sqrt(self.x_var)*torch.sqrt(self.y_var)*torch.div(self.corr, 100))

        # return (torch.logical_and(corr_cons, torch.logical_and(torch.logical_and(x_mean_cons, y_mean_cons), torch.logical_and(x_var_cons, y_var_cons))), self.corr)
        # return (torch.logical_and(corr_cons, torch.logical_and(x_mean_cons, y_mean_cons)), self.corr)
        # return (torch.logical_and(torch.logical_and(corr_cons, x_var_cons), torch.logical_and(x_mean_cons, y_mean_cons)), self.corr)
        # return (torch.logical_and(x_var_cons,torch.logical_and(corr_cons, torch.logical_and(x_mean_cons, y_mean_cons))), self.corr)
        return (torch.logical_and(x_var_cons, corr_cons), self.corr)
prover_gen_settings([x_vals_path, y_vals_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

theory output:  tensor(0.8929)


RuntimeError: std and var only support floating point and complex dtypes

In [24]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


thread '<unnamed>' panicked at src/circuit/ops/region.rs:237:38:
called `Option::unwrap()` on a `None` value


PanicException: called `Option::unwrap()` on a `None` value

In [20]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  2
prf instances:  [[[14955570959218682635, 4667139652385906200, 12836539004462631467, 1774684518626433649], [4224417983558473805, 851357164555783563, 5363851773531956453, 1448631618362554917], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6856486776492523050, 3509301300289033549, 11286023888431465720, 2871037162753880935]]]
proof boolean:  1.0
proof result 1 : 89.0
verified
